In [2]:
from pandas import read_csv
tt = read_csv('tt_hours.csv')
bom = read_csv('bom_hours.csv')

In [3]:
print(bom.keys())
print(tt.keys())

Index(['Lecture', 'Lab', 'Tutorial', 'Other', 'Private', 'Credits',
       'total_hours', 'module'],
      dtype='object')
Index(['day', 'professor', 'module', 'group', 'delivery', 'location',
       'active_weeks', 'start_time', 'end_time', 'course_code', 'course_year',
       'duration', 'occurences', 'total_hours'],
      dtype='object')


In [4]:
tt['group'] = None
timetabled = tt.drop_duplicates(['delivery', 'module'])
timetabled = timetabled.drop(columns=['location', 'group', 'day', 'active_weeks', 'start_time', 'end_time', 'active_weeks', 'total_hours', 'occurences'])


In [5]:

timetabled = timetabled.pivot(index='module', columns='delivery', values='duration')

timetabled = timetabled.fillna(0)
timetabled['LEC'] = timetabled['LEC'].astype(int)
timetabled['TUT'] = timetabled['TUT'].astype(int)
timetabled['LAB'] = timetabled['LAB'].astype(int)

timetabled
# cleaned_df[cleaned_df['module'] == 'FR4626']['duration'].sum()

delivery,LAB,LEC,TUT
module,,,
AC4002,0,1,1
AC4018,0,1,1
AC4024,0,1,1
AC4034,0,1,1
AC4214,0,1,1
...,...,...,...
WT4604,2,1,0
WT4704,0,1,2
WT4804,0,1,0


In [6]:
from pandas import merge

df = merge(left=bom, right=timetabled, how='inner', on='module')
df = df.drop(columns=['Credits', 'total_hours', 'Private'])
df.head(5)


,Lecture,Lab,Tutorial,Other,module,LAB,LEC,TUT
0,1,0,2,0,FR4626,0,1,2
1,2,0,1,0,GE4626,0,2,2
2,2,0,2,0,GA4146,0,1,1
3,3,0,0,0,GA4156,0,1,0
4,3,0,3,0,JA4246,0,1,1


In [7]:
df['tt_p_week'] = df['LEC'] + df['TUT'] + df['LAB']
df['bom_p_week'] = df['Lecture'] + df['Tutorial'] + df['Lab']
df['offset'] = df['tt_p_week'] - df['bom_p_week']
df['offset'] = df['offset'].abs()

conflict_df = df[df['tt_p_week'] != df['bom_p_week']]

conflict_df.index = conflict_df.reset_index().index

conflict_df

,Lecture,Lab,Tutorial,Other,module,LAB,LEC,TUT,tt_p_week,bom_p_week,offset
0,2,0,1,0,GE4626,0,2,2,4,3,1
1,2,0,2,0,GA4146,0,1,1,2,4,2
2,3,0,0,0,GA4156,0,1,0,1,3,2
3,3,0,3,0,JA4246,0,1,1,2,6,4
4,3,0,0,0,CU4026,0,3,2,5,3,2
...,...,...,...,...,...,...,...,...,...,...,...
599,1,0,2,0,GE4628,0,2,2,4,3,1
600,1,0,3,0,GE4248,0,1,1,2,4,2
601,2,0,1,0,PO4048,0,1,1,2,3,1
602,1,0,3,2,FR4248,0,1,1,2,4,2


In [8]:
all_conflicting = df[df['Lecture'] != df['LEC']]
all_conflicting = all_conflicting[all_conflicting['Lab'] != all_conflicting['LAB']]
all_conflicting = all_conflicting[all_conflicting['Tutorial'] != all_conflicting['TUT']]
print('Conflicting Lecture, Lab, Tutorial Hours:', len(all_conflicting))


Conflicting Lecture, Lab, Tutorial Hours: 48


In [9]:
total = len(df)
def compare(label_1='', label_2=''):
    print(label_1)
    incorrect_lec_hours = df[df[label_1] != df[label_2]]
    print(f'Conflicting {label_1} Hours:', len(incorrect_lec_hours), ',', str(round(len(incorrect_lec_hours)/total*100, 2))+'%')

    longer_than_bom = df[df[label_1] < df[label_2]]
    print(f'Longer Timetabled {label_1}s:', len(longer_than_bom), ',', str(round(len(longer_than_bom)/len(incorrect_lec_hours)*100, 2))+'%')
    
    shorter_than_bom = df[df[label_1] > df[label_2]]
    print(f'Shorter Timetabled {label_1}s:', len(shorter_than_bom), ',', str(round(len(shorter_than_bom)/len(incorrect_lec_hours)*100, 2))+'%')

    return (incorrect_lec_hours, longer_than_bom, shorter_than_bom)


In [10]:
incorrect_lec_hours, lec_longer_than_bom, lec_shorter_than_bom = compare('Lecture', 'LEC')
print('\n')
incorrect_lab_hours, lab_longer_than_bom, lab_shorter_than_bom = compare('Lab', 'LAB')
print('\n')
incorrect_tut_hours, tut_longer_than_bom, tut_shorter_than_bom = compare('Tutorial', 'TUT')

Lecture
Conflicting Lecture Hours: 615 , 71.18%
Longer Timetabled Lectures: 74 , 12.03%
Shorter Timetabled Lectures: 541 , 87.97%


Lab
Conflicting Lab Hours: 173 , 20.02%
Longer Timetabled Labs: 65 , 37.57%
Shorter Timetabled Labs: 108 , 62.43%


Tutorial
Conflicting Tutorial Hours: 354 , 40.97%
Longer Timetabled Tutorials: 243 , 68.64%
Shorter Timetabled Tutorials: 111 , 31.36%


In [11]:
""" Incorrect delivery hours but no offsett, shows lecturers reallocating to other types of delivery"""
no_offset = df[df['offset'] == 0]

# incorrect_lec_hours['module'].apply(lambda m: True if(m in no_offset_modules) else False)
from pandas import DataFrame

no_offset_but_incorrect_hours = DataFrame()
for i,row in no_offset.iterrows():
    if(row['module'] in incorrect_lec_hours['module'].tolist()):
        no_offset_but_incorrect_hours = no_offset_but_incorrect_hours.append(row)
    elif(row['module'] in incorrect_tut_hours['module'].tolist()):
        no_offset_but_incorrect_hours = no_offset_but_incorrect_hours.append(row)
    elif(row['module'] in incorrect_lab_hours['module'].tolist()):
        no_offset_but_incorrect_hours = no_offset_but_incorrect_hours.append(row)


In [12]:
ndf = no_offset_but_incorrect_hours

In [15]:
print('diff in bom and tt:', len(set(bom.module.tolist()))-len(set(tt.module.tolist())))
print('len of merged tt and bom:', len(df))
print('modules of no offset:', len(no_offset))
print('modules of no offset but incorrect hours:', len(no_offset_but_incorrect_hours))

print("Of modules with no offset but have conflicting delivery hours (Modules which have equal total hours for the week)")
print("LAB > Lab:", len(ndf[ndf['LAB'] > ndf['Lab']]) )
print("LEC > Lecture:", len(ndf[ndf['LEC'] > ndf['Lecture']]) )
print("TUT > Tutorial:", len(ndf[ndf['TUT'] > ndf['Tutorial']]) )
print('\nWe see of the 143 modules that preserved their bom hours on the timetables, tutorials have become more popular.')

diff in bom and tt: 0
len of merged tt and bom: 864
modules of no offset: 260
modules of no offset but incorrect hours: 143
Of modules with no offset but have conflicting delivery hours (Modules which have equal total hours for the week)
LAB > Lab: 24
LEC > Lecture: 19
TUT > Tutorial: 102

We see of the 143 modules that preserved their bom hours on the timetables, tutorials have become more popular.


In [14]:
offset = df[df['offset'] > 0]
print("Modules with offset:", len(offset))

print("Of modules with no offset but have conflicting delivery hours (Modules which have equal total hours for the week)")
print("LAB > Lab:", len(ndf[ndf['LAB'] > ndf['Lab']]) / len(ndf) * 100)
print("LEC > Lecture:", len(ndf[ndf['LEC'] > ndf['Lecture']]) / len(ndf) * 100)
print("TUT > Tutorial:", len(ndf[ndf['TUT'] > ndf['Tutorial']]) / len(ndf) * 100)
print('\nWe see of the 143 modules that preserved their bom hours on the timetables, tutorials have become more popular.')

Modules with offset: 604
Of modules with no offset but have conflicting delivery hours (Modules which have equal total hours for the week)
LAB > Lab: 16.783216783216783
LEC > Lecture: 13.286713286713287
TUT > Tutorial: 71.32867132867133

We see of the 143 modules that preserved their bom hours on the timetables, tutorials have become more popular.


In [25]:
merge(no_offset[no_offset['LEC'] != no_offset['Lecture']], merge(no_offset[no_offset['LAB'] != no_offset['Lab']], no_offset[no_offset['LEC'] != no_offset['Lecture']], how='left', on='module'), how='left', on='module')

,Lecture,Lab,Tutorial,Other,module,LAB,LEC,TUT,tt_p_week,bom_p_week,...,Lecture_y,Lab_y,Tutorial_y,Other_y,LAB_y,LEC_y,TUT_y,tt_p_week_y,bom_p_week_y,offset_y
0,2,0,1,0,PO4015,0,1,2,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2,0,0,ME4226,2,1,1,4,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,0,1,0,LA4078,0,1,2,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,0,1,0,MA4402,0,1,2,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,0,0,0,WT4704,0,1,2,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,2,0,1,0,PA4018,0,1,2,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
128,2,0,0,0,LI4018,0,1,1,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
129,2,0,1,0,PO4118,0,1,2,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
130,3,0,0,0,CU4018,0,1,2,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
len(no_offset)

260